In [23]:
import numpy as np
import yfinance as yf

np.float_ = np.float64
from prophet import Prophet

import pandas as pd


# Função para coletar dividendos históricos
def coletar_dividendos(ticker):
    acao = yf.Ticker(ticker)
    dividendos = acao.dividends
    dividendos = dividendos.reset_index()
    dividendos.columns = ['ds', 'y']
    dividendos['ds'] = pd.to_datetime(dividendos['ds']).dt.tz_localize(None)
    return dividendos


# Função para coletar dados do índice e calcular a média anual
def calcular_taxa_retorno_bvsp():
    bvsp = yf.Ticker('^BVSP')
    historico = bvsp.history(period='10y')  # Dados dos últimos 10 anos
    historico['Year'] = historico.index.year
    anual = historico.groupby('Year')['Close'].mean()  # Média anual
    retorno_medio_anual = anual.pct_change().mean()  # Taxa de retorno anual média
    return retorno_medio_anual


# Função para calcular a taxa de crescimento dos dividendos usando Prophet
def calcular_taxa_crescimento_dividendos(ticker):
    dividendos = coletar_dividendos(ticker)

    # Criar e ajustar o modelo Prophet
    modelo = Prophet(growth='linear', yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
    modelo.fit(dividendos)

    # Prever para o futuro e obter a taxa de crescimento
    futuro = modelo.make_future_dataframe(periods=1, freq='YE')  # Adiciona um ano
    previsoes = modelo.predict(futuro)
    crescimento_estimado = previsoes['trend'].iloc[-1] - previsoes['trend'].iloc[-2]
    taxa_crescimento = crescimento_estimado / previsoes['trend'].iloc[-2]

    return taxa_crescimento


def calcular_valor_justo(ticker):
    """
    Calcula o valor justo da ação usando o Modelo de Gordon.
    """
    taxa_retorno = calcular_taxa_retorno_bvsp()  # prêmio de 6% acima do IBOV
    taxa_crescimento = calcular_taxa_crescimento_dividendos(ticker)

    print(f'Taxa de retorno média anual do ^BVSP: {taxa_retorno:.4f}')
    print(f'Taxa de crescimento dos dividendos para {ticker}: {taxa_crescimento:.4f}')

    # Coletar o dividendo mais recente
    dividendos = coletar_dividendos(ticker)
    dividendo_atual = dividendos['y'].iloc[-1]
    
    display(dividendos)

    # Calcular o valor justo usando o Modelo de Gordon
    valor_justo = dividendo_atual * (1 + taxa_crescimento) / (taxa_retorno - taxa_crescimento)

    return valor_justo

In [24]:
# Exemplo de uso
ticker = 'B3SA3.SA'  # Exemplo de ticker para dividendos
valor_justo = calcular_valor_justo(ticker)

print(f'O valor justo da ação {ticker} é: {valor_justo:.2f}')

23:27:59 - cmdstanpy - INFO - Chain [1] start processing
23:28:00 - cmdstanpy - INFO - Chain [1] done processing


Taxa de retorno média anual do ^BVSP: 0.0938
Taxa de crescimento dos dividendos para B3SA3.SA: 0.0308


,ds,y
0,2008-08-26,0.024332
1,2009-01-02,0.023102
2,2009-05-04,0.020682
3,2009-05-18,0.018644
4,2009-08-17,0.020019
...,...,...
72,2023-09-29,0.105230
73,2024-01-02,0.100519
74,2024-02-28,0.066900
75,2024-03-27,0.052453


O valor justo da ação B3SA3.SA é: 1.41
